In [1]:
# Import packages
import pandas as pd
import numpy as np
import statistics
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import pycountry_convert as pc
import lightgbm as lgb
from sklearn.linear_model import LinearRegression

# Set options
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
# Import data
train = pd.read_csv("../01-data/train.csv", low_memory = False)
test = pd.read_csv("../01-data/test.csv", low_memory = False)

# Custom data
codebook = pd.read_csv("../01-data/codebook_compact.csv", low_memory = False) # OG codebook+dtypes from codebook_long

In [16]:
train_raw = pd.read_csv("../01-data/train_raw.csv", low_memory = False)

In [5]:
train.head()

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,cntry,satisfied
0,9948,2,2,74,11010,.a,2,2,2,2,1,1,2,2,1,0,0,66,2,2,AT33,2,2,.a,1,66,0,0,0,0,1,0,0,0,0,3,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,2,0,0,3,322,3,322,3,322,.a,.a,10,3,3,3,.a,.a,1,.a,1,1,2,2,9,1,66,2,1,3,4,3,3,2,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,5,2,1,2,3,4,2,2,0,0,1,0,3,3,3,3,2,3,6,3,2,2,2,0,0,NaN,1,1,12,2,4,4,9,25,107.0,2015,2015,4,5,3,4,1,5,3,3,2,2,3,5,2,6,5,3341,.a,1,.a,GER,0,3,2,.a,5,5,66,2,6,.a,1,1,23,20,4,4,2,2,1,1993,0,0,3,10,10,8,1,2,2,2,2,7,1,.a,1,1,.a,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,.a,1,0,2,4,2,1,3,.b,5,1,9,1,4,2015,4,0,5,10,0,2,0,3,0,1,.a,2,.a,0,0,0,0,1,8,40,40,.a,2,2,1,2,2,1941,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,AT,0
1,25601,4,2,58,11010,.a,2,2,2,2,1,2,2,2,2,0,0,66,2,2,AT31,2,2,.a,1,66,0,0,0,0,1,0,0,0,0,4,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,.a,0,0,3,322,2,212,2,212,.a,.a,12,3,2,2,.a,.a,1,.a,1,1,2,2,5,1,66,2,1,1,1,5,3,1,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,4,3,1,2,4,5,2,3,0,0,1,6,3,2,4,3,3,4,4,4,4,1,5,4,2,NaN,17,17,19,15,1,1,17,46,75.0,2015,2015,3,4,3,3,1,1,4,2,2,3,2,2,3,3,1,7132,.a,2,.a,GER,0,5,.b,.a,6,6,66,3,3,.a,1,1,25,.a,8,8,2,2,1,2014,0,0,4,4,7,3,6,.a,4,5,2,5,1,.a,1,1,.a,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,.a,0,0,3,4,2,1,5,3,8,5,8,17,1,2015,4,.b,7,7,.b,5,5,.b,1,3,2,1,1,1,0,0,0,1,3,39,39,.a,2,2,2,2,2,1957,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,AT,0
2,8592,6,2,47,11010,11010,2,2,1,2,1,.a,1,1,2,0,0,66,2,3,AT33,2,.a,.a,1,66,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,2,0,0,2,212,2,212,2,212,3,322,9,2,2,2,3,.a,1,1,1,1,4,1,2,1,66,1,1,1,1,1,3,2,NaN,NaN,NaN,NaN,1,2,2,.a,.a,.a,.a,NaN,10,2,4,2,1,1,1,3,0,0,1,5,2,3,5,2,2,3,5,2,2,2,5,3,6,NaN,18,18,17,28,3,3,16,30,50.0,2015,2015,2,5,2,1,2,5,2,2,2,2,2,5,2,5,4,9112,.d,1,.a,GER,0,4,.d,.a,1,.a,66,3,1,.a,1,1,56,1,1,8,1,2,.a,.a,1,1,3,9,8,8,2,.a,3,2,2,3,1,.a,8,1,.a,NaN,NaN,NaN,NaN,1,2,2,.a,.a,.a,.a,NaN,NaN,NaN,NaN,NaN,1,2,2,.a,.a,.a,.a,NaN,1,0,0,2,5,2,1,6,6,8,6,8,18,3,2015,4,5,9,9,8,5,6,4,5,6,.a,2,.a,0,0,0,0,1,8,30,35,40,4,2,1,2,2,1968,NaN,NaN,NaN,NaN,1963,1993,1995,.a,.a,.a,.a,NaN,AT,1
3,29593,10,2,22,11010,.a,2,2,2,2,1,2,2,2,2,0,0,66,2,10,AT12,2,.a,.a,1,66,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,2,0,0,3,322,3,322,3,322,.a,.a,6,3,3,3,.a,.a,1,.a,1,1,3,2,0,1,66,1,2,2,1,1,1,1,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,8,1,1,2,1,1,2,3,1,0,1,0,4,4,2,2,2,1,3,1,1,4,0,0,3,NaN,7,7,8,46,4,4,7,51,44.0,2015,2015,0,1,4,2,6,3,1,4,1,3,2,3,1,3,6,7231,.a,2,.a,GER,0,10,1,1,4,4,66,3,1,.a,1,1,45,.a,6,5,2,2,.a,.a,1,0,3,5,0,5,7,.a,0,0,2,6,1,.a,5,1,.a,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,NaN,NaN,NaN,NaN,.a,.a,.a,.a,.a,.a,.a,NaN,.a,0,0,3,6,2,1,0,0,0,0,8,7,4,2015,4,0,0,0,0,0,0,0,0,7,.a,2,.a,0,0,0,0,2,0,40,50,.a,3,2,1,2,2,1993,NaN,NaN,NaN,N

In [25]:
train['v5'].value_counts()

.a        19424
.d         1224
12060      1028
25070       705
11010       695
22000       631
11050       592
11030       491
.c          448
13070       407
13071       214
11070       205
15080       186
15110       184
13990       182
15111       175
.b          162
444444      146
13040       137
11031       116
11102       113
15090       111
21220       107
13060        97
11100        94
11081        92
21000        87
22011        83
21150        81
15140        77
11051        73
15020        70
12020        62
11040        57
15130        53
10000        52
21130        50
14100        50
15010        48
13075        46
15060        46
11101        43
14040        41
15040        40
11080        39
11058        36
12040        36
15120        36
13074        35
13010        32
11033        31
11020        31
22013        29
61050        28
14090        25
22012        25
14110        25
15030        25
21010        24
43020        24
23000        24
72010        23
14020   

In [23]:
train_raw['v5'].value_counts()

No second ancestry                                 19424
No answer                                           1224
Swedish nfs                                         1028
Israeli nfs                                          705
Austrian nfs                                         695
Jewish                                               631
French nfs                                           592
British nfs                                          491
Don't know                                           448
Spanish nfs                                          407
Andalusian                                           214
German nfs                                           205
Moravian                                             186
Russian nfs                                          184
South European nec                                   182
Estonian Russian                                     175
Refusal                                              162
Not classifiable               